In [1]:
#! usr/bin/python3
import pandas as pd
import numpy as np
import data_clean as data
import pickle
import re
import nltk
from nltk import wordnet
from nltk import pos_tag
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords 
from data_clean import text_normalization
from sklearn.metrics import pairwise_distances 
import tkinter
from tkinter import *
from nltk.stem import wordnet 
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer 
from nltk import pos_tag 
from nltk import word_tokenize 
from nltk.corpus import stopwords 
import bow
from cosine_similarity import cosine_similarity
import random

In [2]:
nltk.download('punkt')
nltk.download('all')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sapotalg\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\sapotalg\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\sapotalg\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     C:\Users\sapotalg\AppData\Roaming\nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     C:\Users\sapotalg\AppData\Roaming\nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     C:\Users\sapotalg

[nltk_data]    |   Package sinica_treebank is already up-to-date!
[nltk_data]    | Downloading package smultron to
[nltk_data]    |     C:\Users\sapotalg\AppData\Roaming\nltk_data...
[nltk_data]    |   Package smultron is already up-to-date!
[nltk_data]    | Downloading package state_union to
[nltk_data]    |     C:\Users\sapotalg\AppData\Roaming\nltk_data...
[nltk_data]    |   Package state_union is already up-to-date!
[nltk_data]    | Downloading package stopwords to
[nltk_data]    |     C:\Users\sapotalg\AppData\Roaming\nltk_data...
[nltk_data]    |   Package stopwords is already up-to-date!
[nltk_data]    | Downloading package subjectivity to
[nltk_data]    |     C:\Users\sapotalg\AppData\Roaming\nltk_data...
[nltk_data]    |   Package subjectivity is already up-to-date!
[nltk_data]    | Downloading package swadesh to
[nltk_data]    |     C:\Users\sapotalg\AppData\Roaming\nltk_data...
[nltk_data]    |   Package swadesh is already up-to-date!
[nltk_data]    | Downloading package swi

True

In [3]:
pickle_name='posts.pkl'
#C:\Users\sapotalg\Desktop\Swati\DataScience\TherapistBot\corpora
df=pd.read_csv(r'C:\Users\sapotalg\Desktop\Swati\DataScience\TherapistBot\Scrapped_Data.csv')
df['lemmatized_text']=df['questionText'].apply(data.text_normalization)
df.to_pickle('./pickle_dataframe/%s' % pickle_name)
df

,Unnamed: 0,questionID,questionTitle,questionText,questionLink,topic,therapistInfo,therapistURL,answerText,upvotes,views,split,lemmatized_text
0,0,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,"Sherry Katz, LCSWCouples and Family Therapist,...",https://counselchat.com/therapists/sherry-katz...,"If everyone thinks you're worthless, then mayb...",1,2899,train,im go through some thing with my feeling and m...
1,1,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,"Robin Landwehr, DBH, LPCC, NCCMental Health in...",https://counselchat.com/therapists/robin-landw...,"Hello, and thank you for your question and see...",1,3514,train,im go through some thing with my feeling and m...
2,2,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,Lee KingI use an integrative approach to treat...,https://counselchat.com/therapists/lee-king,First thing I'd suggest is getting the sleep y...,0,5,train,im go through some thing with my feeling and m...
3,3,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,"Shauntai Davis-YearginPersonalized, private on...",https://counselchat.com/therapists/shauntai-da...,Therapy is essential for those that are feelin...,0,31,train,im go through some thing with my feeling and m...
4,4,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,Jordan WhiteLicensed Social Worker at Oak Root...,https://counselchat.com/therapists/jordan-white,I first want to let you know that you are not ...,0,620,train,im go through some thing with my feeling and m...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2124,2124,20,What happens in a counseling session?,"After first meeting the client, what is the pr...",https://counselchat.com/questions/what-happens...,counseling-fundamentals,Victoria HaagFind direction for your life...,https://counselchat.com/therapists/victoria-haag,There are probably no two therapists alike bec...,0,45,train,after first meet the client what be the proces...
2125,2125,20,What happens in a counseling session?,"After first meeting the client, what is the pr...",https://counselchat.com/questions/what-happens...,counseling-fundamentals,Allison VelezIs your relationship worth it?,https://counselchat.com/therapists/allison-velez,"Each counselor may have a different process, s...",0,37,train,after first meet the client what be the proces...
2126,2126,20,What happens in a counseling session?,"After first meeting the client, what is the pr...",https://counselchat.com/questions/what-happens...,counseling-fundamentals,David RouttPresident and Clinical Counselor at...,https://counselchat.com/therapists/david-routt,"After meeting a client, many Counselors will a...",0,56,train,after first meet the client what be the proces...
2127,2127,20,What happens in a counseling session?,"After first meeting the client, what is the pr...",https://counselchat.com/questions/what-happens...,counseling-fundamentals,2nd Chance Counseling Service Online Addiction...,https://counselchat.com/therapists/2nd-chance-...,A good therapist will discuss what brought you...,0,27,train,after first meet the client what be the proces...


In [4]:
#Greeting

GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey", "how are you","How are you")
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me","Thanks for asking"]
def greeting(sentence):
 
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [5]:
#Data pre-processing 

def text_normalization(text):
    text=str(text).lower()
    spl_char_text=re.sub(r'[^ a-z]', '', text)
    tokens=nltk.word_tokenize(spl_char_text)
    lema=wordnet.WordNetLemmatizer()
    tags_list=pos_tag(tokens, tagset=None)
    lema_words=[]
    for token,pos_token in tags_list:
        if pos_token.startswith('V'):
            pos_val='v'
        elif pos_token.startswith('J'):
            pos_val='a'
        elif pos_token.startswith('R'):
            pos_val='r'
        else:
            pos_val='n'
        lema_token=lema.lemmatize(token, pos_val)
        lema_words.append(lema_token)

    return " ".join(lema_words)

In [6]:
# Cosine Similarity for user question
def cosine_similarity(df_bow, question_bow):
    cosine_value = 1- pairwise_distances(df_bow, question_bow, metric = 'cosine' )
    return cosine_value


In [7]:
# Bag Of Words(BOW) implementation using Lemmitized_text
def context_bow(df):
    cv = CountVectorizer()
    X = cv.fit_transform(df['lemmatized_text']).toarray()
    features = cv.get_feature_names()
    df_bow = pd.DataFrame(X, columns = features)
    return df_bow, cv

def user_question_bow(user_question, cv):
    stop = stopwords.words('english')
    # check for stop words
    Q=[]
    a=user_question.split()
    for i in a:
        if i in stop:
            continue
        else:
            Q.append(i)
        b=" ".join(Q)
    question_lemma = text_normalization(b)
    #applying bow
    question_bow = cv.transform([question_lemma]).toarray() 
    return question_bow

In [8]:
pickle_name='posts.pkl'
def main():
    df = pd.read_pickle(r'C:\Users\sapotalg\Desktop\Swati\DataScience\TherapistBot\pickle_dataframe\posts.pkl')
    print('read pickle')
    print('getting df_bow')
    df_bow, cv=bow.context_bow(df)
    print('done df_bow\nWaiting on a question')
    
    def send():
        msg = EntryBox.get("1.0",'end-1c').strip()
        EntryBox.delete("0.0",END)
        continue_discussion=True
        
        if(msg.casefold() !='bye'):
                if(msg.casefold() =='thanks' or msg.casefold() =='thank you very much'  or msg.casefold() =='thank you'):
                    print("Chatbot: Most welcome")
                    ChatLog.config(state=NORMAL)
                    ChatLog.insert(END, "You: " +msg+ '\n\n')
                    ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
                    ChatLog.insert(END, "Therapist Bot:  Most welcome .."+'\n\n')
                    ChatLog.config(state=DISABLED)
                    ChatLog.yview(END)
        
                else:
                    if(greeting(msg)!=None):
                        ChatLog.config(state=NORMAL)
                        ChatLog.insert(END, "You: " +msg+ '\n\n')
                        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
                        res=greeting(msg)
                        ChatLog.insert(END, "Therapist Bot: "+res+ '\n\n')
                        ChatLog.config(state=DISABLED)
                        ChatLog.yview(END)
                    else:
                        print("Chatbot: ",end="")
                        ChatLog.config(state=NORMAL)
                        ChatLog.insert(END, "You: " + msg + '\n\n')
                        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
                        question_bow=bow.user_question_bow(msg, cv)
                        print('done question bow')
                        # create cosine similarity
                        print('implimenting cosine similarity')
                        df['similarity_bow']=cosine_similarity(df_bow, question_bow)
                        print('cosine similarity completed')
                        print('doing similarity')
                        df_simi = pd.DataFrame(df, columns=['questionText', 'answerText','similarity_bow'])
                        print('done with similarity')
                        # sort to get answer for similar questions
                        print('doing  similarity_sorting')
                        df_simi_sort = df_simi.sort_values(by='similarity_bow', ascending=False)
                        print('completed similarity_sorting')
                        print('sending text response')
                        res = df_simi_sort.iloc[0]['answerText']
                        print('text response sent')
                        print(df_simi_sort.iloc[0]['questionText'])
                        ChatLog.insert(END, "Therapist Bot: " + res + '\n\n')
                        ChatLog.config(state=DISABLED)
                        ChatLog.yview(END)  
                        
               
                                            
        else:
                continue_discussion=False
                print("Chatbot: Take care, bye ..")
                ChatLog.config(state=NORMAL)
                ChatLog.insert(END, "You: " +msg+ '\n\n')
                ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
                ChatLog.insert(END, "Therapist Bot:  Take care, bye .."+'\n\n')
                ChatLog.config(state=DISABLED)
                ChatLog.yview(END)
        


    base = Tk()
    base.title("Therapist Bot.")
    base.geometry("400x500")
    base.resizable(width=FALSE, height=FALSE)

    #Create Chat window
    ChatLog = Text(base, bd=0, bg="white", height="20", width="200", font="Arial",)


    #Bind scrollbar to Chat window
    scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
    ChatLog['yscrollcommand'] = scrollbar.set

    #Create the box to enter message
    EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
    EntryBox.bind("<Return>", send)

    #Create Button to send message
    SendButton = Button(base, bg='green', font=("Verdana",12,'bold'), text="Send", width="8", height=5,
                        bd=0, fg='black', command= send )


    #Place all components on the screen
    scrollbar.place(x=376,y=6, height=386)
    ChatLog.place(x=6,y=6, height=386, width=370)
    EntryBox.place(x=128, y=401, height=90, width=265)
    SendButton.place(x=6, y=401, height=90)
    ChatLog.insert(END, "Therapist Bot: Welcome!My name is Therapist Bot. How can I help you today?" + '\n\n')

    ChatLog.config(state=DISABLED)
    base.mainloop()
    #####

if __name__ == '__main__':
    main()



read pickle
getting df_bow
done df_bow
Waiting on a question
